In [1]:
import pandas as pd
import plotly.express as px
# Install & Import Dependencies
import wandb

pd.options.plotting.backend = "plotly"

In [2]:
# Define WandB Parameters
entity = "phys_inversion"
projects = ["eval_results_direction_facing_gravity", "eval_results_reach_gravity"]
filters = {"state": {"$ne": "finished"}}
api = wandb.Api()

In [3]:
# Fetch Data from WandB

def get_runs(entity, project, filters):
    runs = api.runs(f"{entity}/{project}", filters=filters)
    data = []
    for run in runs:
        history = run.history(keys=None, pandas=True)  # Adjust as needed

        # Fetch hyperparameters
        config = run.config
        history = pd.concat([history, pd.json_normalize(config, sep='.')], axis=1)

        history["run_name"] = run.name
        history["run_id"] = run.id

        data.append(history)
    return pd.concat(data).reset_index() if data else pd.DataFrame()


In [4]:
gpu_ids = [2]
cmds = []
for project in projects:
    df = get_runs(entity, project, filters)
    perturbations_cols = [x for x in df.columns if 'perturbations' in x]
    perturbations = df[perturbations_cols]
    for _, row in df.iterrows():
        re_run_string = (
            f"python phys_anim/eval_checkpoints.py"
            f" +checkpoint_paths=[{row['checkpoint']}]"
            f" +gpu_ids={gpu_ids}"
            f" +wandb.project={project}"
            f" +use_perturbations=True"
        )
        for perturbation in perturbations:
            re_run_string += f" +{perturbation.replace('env.config.', '')}={row[perturbation]}"
        cmds.append(re_run_string)

In [5]:
with open("../re_run.sh", "w") as f:
    for cmd in cmds:
        f.write(f"{cmd}\n")